# Export Psi4 as OLDMOS

* Run Psi4
* Convert MOs (reordering, scaling)
* Export as `PSIMOS`

In [2]:
import psi4
import numpy as np
import sys
sys.path.append('../lib')
from P4toC4_aux import *

Run Psi4 with Cartsian coordiantes that will **not**
be reoriented in either Psi4 or CFOUR.

The original input was:

```
0 2
O
H 1 0.97
```

In [3]:
psi4.set_memory('500 MB')
psi4.core.set_global_option("BASIS", "cc-pvdz")
psi4.core.set_global_option("SCF_TYPE", "pk")
psi4.core.set_global_option("D_CONVERGENCE", 1e-8)
psi4.core.set_global_option("PUREAM", "True")
psi4.core.set_global_option("REFERENCE", "UHF")
psi4.core.set_output_file('output.dat', False)

h2o = psi4.geometry("""
0 2
O   0.0   0.0  -0.057496044831
H   0.0   0.0   0.912503955169
symmetry c1
""")

E, wf = psi4.energy('scf', return_wfn=True)
E

-75.3938389265448

In [4]:
# Use this as GENBAS to avoid issues with
# generalized vs segemented contraction.
# 
basisset=wf.basisset()
#print(basisset.genbas())

From the basisfunction object, derive the mapping and scaling for the operation:
```
Cfour_MO[mapping[i]] = Psi4_MO[i] / scale[i]
```
and apply it.

In [5]:
mapping, scale =basis_mapping(wf.basisset(), verbose=0)
Ca=np.array(wf.Ca())
Cb=np.array(wf.Cb())
Ca_C4 = psi4_to_c4(Ca, mapping, scale)
Cb_C4 = psi4_to_c4(Cb, mapping, scale)
write_oldmos('PSIMOS', Ca_C4, Cbs=Cb_C4)

Move the file PSIMOS to your CFOUR work-directory
(for this example, `Cfour/UHF`), and run the script `runscf`.